# QuDiPy tutorial: Using the Stark shift module

This tutorial will show how to use the Strak shift module to determine the deviation in $g$-factor arising from an applied electric field in the z-direction, where the plane of a device would be in the xy-plane.

In this tutorial, we will be defining the potential and the electric field that the dot(s) reside in analytically and therefor the xy-confining potential and the z-direction electric field will be decoupled. However, in practice with NextNano potential, these two values will not be decoupled. The goal is to make this tutorial with NextNano potentials in due time,

In order for the file to not be too large, the output of the cells has been cleared so you will need to run the cells on your local machine to see the output.

## 1. Add the current location to path
In order for this tutorial to work, you will need to run this from the tutorials folder.

In [ ]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))

## 2. Import the relevant modules and set plotting parameters

In [ ]:
# From module
import qudipy as qd
import qudipy.potential as pot
import qudipy.qutils as qt
import qudipy.starkshift as ss

# From external libraries
import numpy as np
import matplotlib.pyplot as plt

## 3. Single dot 
### 3.a) Define the potential where dots will be located

First, we must define the xy-potential where the dots will reside. For this tutorial, we will have a single dot in harmonic potential. This potential is analytically defined, but any potential can be used.

In [ ]:
consts=qd.Constants("Si/SiO2")

# Create control voltage names and variables the same way you would normally
ctrl_names = ['V1']

V1 = [0.2, 0.22, 0.24, 0.26, 0.28]

ctrl_vals = [V1]

# Specify the x and y points you would like to have in a grid
x = np.arange(-60e-9, 60e-9, 1e-9)
y = np.arange(-60e-9, 60e-9, 1e-9)

# Create the function which defines the potential, with the control variables as the first argument and
# a GridParameters object as the second argument

def create_potentials(cvals, gparams):
    # Get coordinates from mesh
    x_pot = gparams.x_mesh
    y_pot = gparams.y_mesh

    # Get voltages out from the control valriables
    v_1 = cvals[0]

    # Convert (linear conversion) from volrages to chemical potentials
    # This is just a linear relationship
    lever_arm_1 = 1
    mu_1 = v_1 * lever_arm_1 * consts.e

    # Define frequency of harmonic oscillators (picked for )
    omega_1 = 5e12
    
    # Define the potential
    potential = 1/2 * consts.me * omega_1**2 * (np.square(x_pot) + np.square(y_pot)) - mu_1

    return potential

# Create the analytical potential using the variables defined above
analytic_potential = pot.load_data.analytical_potential(ctrl_vals, ctrl_names, create_potentials, x, y)

# Build the interpolator
pot_interp = pot.build_interpolator(analytic_potential, constants=qd.Constants("Si/SiO2"))

# Now specify the potentials to interpolate around
v_vec = [0.25]

# Return the potential interpolated (or calculated) at that particular value
new_pot = pot_interp(v_vec)

# Plot the 2D potential
pot_interp.plot(v_vec)


### 3.b) Solve Schrodinger equation in potential

In [ ]:
gparams = pot.GridParameters(x, y)
gparams.update_potential(new_pot)

e_ens, e_vecs = qt.solvers.solve_schrodinger_eq(consts, gparams, n_sols=1)

ground_state = e_vecs[:,:,0]

fig, axes = plt.subplots(1, 1, figsize=(10, 6))

fig.subplots_adjust(hspace=0.4, wspace=0.25)

axes.imshow(np.real(np.multiply(ground_state, ground_state.conj())), cmap='viridis', 
            extent=[gparams.x.min()/1E-9, gparams.x.max()/1E-9,
                    gparams.y.min()/1E-9, gparams.y.max()/1E-9])
axes.set_title("State: "+str(0))
axes.set(xlabel='x-coords [nm]',ylabel='y-coords [nm]')

plt.show()

### 3.c) Define electric field

In [ ]:
e_field = np.full(ground_state.shape, 28e6)
delta_g = ss.starkshift.delta_g(gparams, ground_state, e_field)
print(delta_g)

### 3.d) Calculate expectation value of field